In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from collections import Counter
from konlpy.tag import Okt
import re
import os

okt=Okt()

data = pd.read_csv('../data/최종_전세사기_all_.csv') #원본 데이터
#tf불러오기
with open('../data/title_tf_df.pickle', 'rb') as f:
    tf = pickle.load(f)
#idf불러오기
with open('../data/title_idf.pickle', 'rb') as f:
    idf = pickle.load(f)
#tfidf불러오기
with open('../data/title_tfidf_df.pickle', 'rb') as f:
    tfidf = pickle.load(f)
    
class Similarity():
   
    def __init__(self,new_text):
        self.new_text=new_text

    def view_similarity3(self):
        # 새로운 글 명사 추출 후 TF구하기
        self.new_nouns=okt.nouns(self.new_text) # new_text 명사추출
        self.new_tf=pd.Series(Counter(self.new_nouns)) # new_text TF구하기
        self.new_text_filter=[i for i in self.new_tf.index if i in idf.index] # 새로 생긴 단어(idf에 없는 단어) 없애기
        self.new_tf=self.new_tf[self.new_text_filter]
        # 새로운 글 tfidf구하기
        self.new_tfidf=self.new_tf*idf
        self.new_tfidf=self.new_tfidf.fillna(0)
        self.new_tfidf=self.new_tfidf.values.reshape(1,-1)

        # 새로운 글과 가장 유사한 원본데이터(data)의 행 찾기
        self.sim_res=[cosine_similarity(self.new_tfidf,tfidf.loc[i].values.reshape(1,-1)).flatten().tolist() for i in tfidf.index]
        self.sim_res_dict=[(idx,i[0]) for idx, i in enumerate(self.sim_res)]
        self.sim10_sort=sorted(self.sim_res_dict, key=lambda x: x[1], reverse=True)[:10] # 유사한 문서 10개 (행번호,코사인유사도 값)
        self.sim10_sort_index=[i[0] for i in self.sim10_sort] #  유사한 행번호 10개, 내림차순 정렬
        self.sim10_sort_score=[i[1] for i in self.sim10_sort] #  유사도 점수 10개, 내림차순 정렬
        #저장
        filename=' '.join(re.findall('[가-힣0-9a-zA-Z]+',self.new_text)) #파일이름(한글,숫자, 영어)로 제한, 특수문자 제외
        data.iloc[self.sim10_sort_index].to_csv(f'./data/제목_similarity10_{filename}.csv')
        print(f'"{os.getcwd()}"폴더에 유사한 글 10개가 "./data/제목_similarity10_{filename}.csv" 파일명으로 저장되었습니다.')
        
        # 새로운글과 유사한 문장 3개 보여주기
        self.df=data.loc[self.sim10_sort_index]
        print('==============================================================================================')
        print(f'   >>> 질문(Q) : {self.new_text}')
        print('==============================================================================================')
        for idx,i in enumerate(self.df.index[:3]):
            self.date=self.df['날짜'][i]
            print(f'\n▶ 제목({idx+1}): 유사도({self.sim10_sort_score[idx]}), <날짜:{self.date}>')
            print(self.df['title'][i])
            print(f'\n▶ 내용({idx+1}):')
            print(self.df['내용'][i])
            print(f'\n▶ 응답({idx+1}):')
            print(self.df['내용1(응답)'][i])
            print('-'*100)
        
if __name__=='__main__':
    Q='갭투자 사기 당한 것 같아요'
    s_text=Similarity(Q)
    s_text.view_similarity3()

"C:\Users\hmh\Documents\py\_프로젝트\프로젝트_교육생\문서유사도"폴더에 유사한 글 10개가 "./data/제목_similarity10_갭투자 사기 당한 것 같아요.csv" 파일명으로 저장되었습니다.
   >>> 질문(Q) : 갭투자 사기 당한 것 같아요

▶ 제목(1): 유사도(0.7134522605629661), <날짜:nan>
전세 갭투자 사기 일까요?

▶ 내용(1):
nan

▶ 응답(1):
         어느 정도의 돈이 있지 않으면 경매나 매매로는 매수하기 어렵습니다.지금처럼 심할 정도로 대출 규제를 하고 있는 상황에서는 2천으로는 도전 자체가 어렵습니다.경매로 취득할 경우에는 낙찰자가 전세금을 모두 인수하는 조건으로 낙찰 받아야 하는데 그러기는 좀 어려울 것입니다.매매 물건으로 매수하는 경우에도 1억도 안 되는 싼 빌라 등이면 어느 정도 가능하지만,그래도 전세금은 8천 이상은 되어 있어야 갭투자 가능한데 그러기도 좀 어렵습니다.최소 5천 ~ 1억 정도는 있을 때에 도전해볼만한 물건이 나옵니다.경매보다는 매매가 더 가능성이 큽니다.     
----------------------------------------------------------------------------------------------------

▶ 제목(2): 유사도(0.7134522605629661), <날짜:2022.05.09>
갭투자 전세사기

▶ 내용(2):
말그대로 입니다.빌라한동 8채중 5채가 같은집주인이고최근에서야 세무서 압류 당한사실을 알았어요.얼마전 매스컴에 나온 1200채 갭투자 한사람이 집주입니다.궁굼한건 경매나 공매진행 될때입니다.전세금 2억이라 했을경우1.전입신고후 확정일자 받았을때2.1+전세금반환보험가입 했을때 입니다.우선순위와 받을수 있는 전세금이 궁굼합니다.

▶ 응답(2):
         법무사입니다.만약 근정당에 비해서 보증금약이 후순위라면 보증금액을 반환받기 힘들수 있습니다.     
-----------------